### Transformer 理论模型

本笔记本存储了关于 Transformer 的一系列分析，例如估算浮点运算次数（FLOPs）、参数数量、峰值内存占用、检查点大小等。

In [1]:
from collections import OrderedDict

In [2]:
# config_args = {
#     'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
#     'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
#     'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
#     'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
# }[model_type]

block_size = 1024
vocab_size = 50257
n_layer = 12
n_head = 12
n_embd = 768
bias = False
assert not bias, "this notebook assumes bias=False just for simplicity"

In [ ]:
def params():
    """估算模型中的参数数量"""
    out = OrderedDict()

    # 令牌和位置嵌入
    out['emebedding/position'] = n_embd * block_size
    out['embedding/token'] = n_embd * vocab_size
    out['embedding'] = out['emebedding/position'] + out['embedding/token']

    # 注意力块
    out['attention/ln'] = n_embd # 注意，我们的LN中bias=False
    out['attention/kqv'] = n_embd * 3*n_embd
    out['attention/proj'] = n_embd**2
    out['attention'] = out['attention/ln'] + out['attention/kqv'] + out['attention/proj']

    # MLP块
    ffw_size = 4*n_embd # 前馈层大小
    out['mlp/ln'] = n_embd
    out['mlp/ffw'] = n_embd * ffw_size
    out['mlp/proj'] = ffw_size * n_embd
    out['mlp'] = out['mlp/ln'] + out['mlp/ffw'] + out['mlp/proj']
    
    # Transformer和其余部分
    out['block'] = out['attention'] + out['mlp']
    out['transformer'] = n_layer * out['block']
    out['ln_f'] = n_embd # 最终层归一化
    out['dense'] = 0 # 由于参数共享为0。该层使用嵌入层的权重

    # 总计
    out['total'] = out['embedding'] + out['transformer'] + out['ln_f'] + out['dense']

    return out

# 将我们的参数计数与PyTorch报告的进行比较
p = params()
params_total = p['total']
print(f"我们看到: {params_total}, 预期: {124337664}, 匹配: {params_total == 124337664}")
# 创建表头
print(f"{'名称':20s} {'参数':10s} {'比例 (%)':10s}")
for k,v in p.items():
    print(f"{k:20s} {v:10d} {v/params_total*100:10.4f}")

In [ ]:
# 现在我们可以计算每个检查点的大小
# 参数以fp32存储，AdamW优化器为每个参数有2个额外的缓冲区用于统计信息
params_bytes = params_total*4
params_and_buffers_bytes = params_bytes + 2*params_bytes
print(f"估计检查点大小: {params_and_buffers_bytes/1e9:.2f} GB")
measured_bytes = 1542470366 # 来自 wc -c ckpt.pt
print(f"用wc -c ckpt.pt测量: {measured_bytes}")
print(f"冗余比率: {measured_bytes/params_and_buffers_bytes*100:.2f}%")

我们还可以估算GPU内存中仅被权重和AdamW优化器内部缓冲区占用的比例

In [ ]:
gpu_memory = 40e9 # 40 GB A100 GPU，大约
print(f"仅参数占用的内存比率: {params_and_buffers_bytes / gpu_memory * 100:.2f}%")

即对于这个小模型来说，内存占用并不多，大部分内存是激活值（前向和反向传播）。当然，对于越来越大的模型，这种情况会发生显著变化。

让我们估算单次前向传播的FLOPs。

In [ ]:
def flops():
    # 我们只计算权重FLOPs，所有其他层（LayerNorm、Softmax等）实际上可以忽略不计
    # 我们计算实际的FLOPs，而不是MACs。因此到处都是2*
    # 基本上对于任何矩阵乘法 A (BxC) @ B (CxD) -> (BxD)，浮点运算次数为 2*B*C*D

    out = OrderedDict()
    head_size = n_embd // n_head

    # 注意力块
    # 1) 投影到键、查询、值
    out['attention/kqv'] = 2 * block_size * (n_embd * 3*n_embd)
    # 2) 计算注意力分数
    out['attention/scores'] = 2 * block_size * block_size * n_embd
    # 3) 值的归约 (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
    out['attention/reduce'] = 2 * n_head * (block_size * block_size * head_size)
    # 4) 最终的线性投影
    out['attention/proj'] = 2 * block_size * (n_embd * n_embd)
    out['attention'] = sum(out['attention/'+k] for k in ['kqv', 'scores', 'reduce', 'proj'])

    # MLP块
    ffw_size = 4*n_embd # 前馈层大小
    out['mlp/ffw1'] = 2 * block_size * (n_embd * ffw_size)
    out['mlp/ffw2'] = 2 * block_size * (ffw_size * n_embd)
    out['mlp'] = out['mlp/ffw1'] + out['mlp/ffw2']

    # Transformer和其余部分
    out['block'] = out['attention'] + out['mlp']
    out['transformer'] = n_layer * out['block']
    out['dense'] = 2 * block_size * (n_embd * vocab_size)

    # 前向、反向、总计
    out['forward_total'] = out['transformer'] + out['dense']
    out['backward_total'] = 2 * out['forward_total'] # 使用常见估计：反向=2*前向
    out['total'] = out['forward_total'] + out['backward_total']

    return out
    
# 将我们的参数计数与PyTorch报告的进行比较
f = flops()
flops_total = f['forward_total']
print(f"{'名称':20s} {'浮点运算':14s} {'比例 (%)':10s}")
for k,v in f.items():
    print(f"{k:20s} {v:14d} {v/flops_total*100:10.4f}")

In [ ]:
# 现在这是从PaLM论文复制粘贴的估算
# 这个公式通常用于计算MFU（模型浮点运算利用率）
def palm_flops():
    """根据PaLM论文公式估算模型浮点运算"""
    # 非嵌入模型参数。注意我们不减去
    # embedding/token参数，因为这些是共享的并在最后一层中使用。
    N = params()['total'] - params()['emebedding/position']
    L, H, Q, T = n_layer, n_head, n_embd//n_head, block_size
    mf_per_token = 6*N + 12*L*H*Q*T
    mf = mf_per_token * block_size
    return mf

print(f"palm_flops: {palm_flops():d}, flops: {flops()['total']:d}, 比率: {palm_flops()/flops()['total']:.4f}")

好的，它们非常相似，这让我对flops()函数中的数学计算有了一些信心。现在，A100在张量核心上的bfloat16性能被标注为312TFLOPS。那么我们的模型浮点运算利用率（MFU）是多少？我训练上述模型时使用batch_size为20，grad_accum为5，在单个A100 GPU上大约运行755ms。我们得到：

In [ ]:
# 这是我们当前大致测量到的
batch_size = 20 * 5 # 5是grad_accum，所以总批次大小是100
measured_time = 0.755 # 每次迭代的秒数
measured_throughput = batch_size / measured_time
flops_achieved = f['total'] * measured_throughput

# A100被标注为在张量核心上运行bfloat16时为312 TFLOPS
a100_flops_promised = 312e12

# 我们正在使用的A100的比例：
print(f"A100使用率: {flops_achieved / a100_flops_promised * 100:.2f}%")

作为参考，我们希望在50%+左右，而且不仅仅是单个GPU，而是整个DDP运行。所以我们还有一些工作要做，但至少我们在这个GPU可实现性能的约2倍范围内。

In [ ]:
# 最后让我们检查6ND近似作为训练的总FLOPs成本
model_size = params()['total'] # 这是参数数量，N
tokens_num = 300e9 # 3000亿令牌，这是数据集大小（以令牌为单位），D
a100_flops = 312e12 # 312 TFLOPS
assumed_mfu = 0.3 # 假设这个模型浮点运算利用率（取上面当前的37%并加上一些DDP开销）
flops_throughput = a100_flops * 8 * assumed_mfu # 假设一个8xA100节点在30%利用率
flops_needed = 6 * model_size * tokens_num # 6ND
time_needed_s = flops_needed / flops_throughput # 以秒为单位
print(f"训练模型所需时间: {time_needed_s/3600/24:.2f} 天")

这个估算相当不错。我训练了这个模型，它在大约4天内收敛。顺便说一句，作为6ND来源的良好参考以及对它的一些直觉，我推荐 [Dzmitry的文章](https://medium.com/@dzmitrybahdanau/the-flops-calculus-of-language-model-training-3b19c1f025e4)。

现在，FLOPs只是一个约束，我们必须密切关注的另一个约束是内存带宽。TODO 稍后估算我们模型的LOAD/STORE成本。